In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from collections import OrderedDict
import joblib

data = pd.read_csv('summary_all.txt', header = 0, sep = '\t')
data = data[(data['maxUF_S_comb']>0) & (data['maxUF_S_comb']>0)]
data.drop(columns=['density_product','tolerance_hub1_rotat3','tolerance_hub2_rotat3'],inplace=True)

X = data.iloc[:,1:-2]
y = data.iloc[:,-2:]


X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.1, random_state=42)

X_scaler = MinMaxScaler()
X_train_scaled = X_scaler.fit_transform(X_train_raw)
X_test_scaled = X_scaler.transform(X_test_raw)

y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train_raw)
y_test_scaled = y_scaler.transform(y_test_raw)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test_scaled)


# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [15]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs
    gpu_count = torch.cuda.device_count()
    print(f"Number of GPUs: {gpu_count}")
    
    # Get the name of each GPU
    for i in range(gpu_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    
    # Get current GPU device id
    current_device = torch.cuda.current_device()
    print(f"Current GPU device id: {current_device}")
    
    # Get memory information
    print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

CUDA available: False


In [16]:
import subprocess

try:
    # Run nvidia-smi command
    nvidia_smi_output = subprocess.check_output("nvidia-smi", shell=True)
    print(nvidia_smi_output.decode())
    print("GPU is available for training")
except:
    print("nvidia-smi command failed - GPU may not be available")

Thu May  1 15:32:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 571.96                 Driver Version: 571.96         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4500             WDDM  |   00000000:AC:00.0  On |                  Off |
| 30%   38C    P8             24W /  200W |    1865MiB /  20470MiB |     13%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [17]:
# Check your PyTorch build
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch built with CUDA: {torch.backends.cudnn.enabled if torch.cuda.is_available() else 'No - CPU only'}")

PyTorch version: 2.5.1+cpu
PyTorch built with CUDA: No - CPU only
